In [1]:
import os
import cv2
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

In [2]:
df = pd.read_csv('한국농수산식품유통공사_전통주 정보_20230301.csv', encoding='cp949')
print(df.iloc[:10, 0])
search_list = df.iloc[:10, 0].tolist()  # 첫 10개 검색어만 사용
print(search_list)
driver = webdriver.Chrome()
driver.get('https://www.sooldamhwa.com')

# 술 리스트
liquor_list = []

0           입장탁주
1        톡 한잔 소주
2    자연담은 복분자막걸리
3        로얄 안동소주
4           지란지교
5    지란지교 무화과 탁주
6        지란지교 약주
7           꽃 와인
8      인사이더 블루베리
9     로제사이더 오미한잔
Name: 전통주명, dtype: object
['입장탁주', '톡 한잔 소주', '자연담은 복분자막걸리', '로얄 안동소주', '지란지교', '지란지교 무화과 탁주', '지란지교 약주', '꽃 와인', '인사이더 블루베리', '로제사이더 오미한잔']


In [3]:
def search(name):
    time.sleep(3)
    # homepage 들어가기
    homepage = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[1]/div/section[1]/ul/li[3]/div/span')
    homepage.click()
    
    # 검색창 클릭
    time.sleep(1)
    input_box = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div[1]/div/form/input')
    input_box.clear()
    input_box.send_keys(name)
    input_box.send_keys(Keys.RETURN)
    
    found_drinks = []
    try:
        element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/div/div/div/div[2]/div/div[1]/div/a'))
        WebDriverWait(driver, 3).until(element_present)
        
        for idx in range(1, 31):  # 시작 인덱스를 1로 변경 (1부터 30까지 검사)
            # 예상 검색 결과 요소 선택
            result_element = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/div/div/div/div[2]/div/div[{}]/div/a'.format(idx))
            
            if "선물" in result_element.text:
                print("선물은 배제됩니다.")
                print("=" * 50)
                continue  # 다음 술을 검색합니다.
            else:
                # 검색 결과 링크 클릭
                result_element.click()
                found_drinks.append(name)
                liquor_list.append(name)
                # print(f"찾은 술: {name}")
                return result_element, found_drinks

    except TimeoutException:
        print(f"검색 결과가 없습니다: {name}")
        print("=" * 50)
        return []
    except NoSuchElementException:
        pass  # 해당 요소를 찾지 못해도 넘어갑니다.
    
    # 검색 결과가 없는 경우 빈 리스트 반환
    if not found_drinks:
        return None
    return found_drinks, result_element
            


In [4]:
# for index, name in search_list:
#     print(f"{name}을(를) 찾고있습니다.")
#     found_drinks = search(driver, name)
#     liquor_list.extend(found_drinks)
    
# print("Found drinks:")
# print("총 {}개 입니다.".format(len(liquor_list)))
# print(liquor_list)

In [5]:
# 주종
def type_liquor():
    type_liquor_element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[3]/div[1]/span'))
    )
    type_liquor_text = type_liquor_element.text
    # print("주종:", type_liquor_text)
    df.at[idx, '주종'] = type_liquor_text
    return type_liquor_text

# 도수
def number_alcohol():
    number_alcohol_element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[3]/div[2]/span'))
    )
    number_liquor_text = number_alcohol_element.text
    # print("도수:", number_liquor_text)
    df.at[idx, '도수'] = number_liquor_text
    return number_liquor_text

# 유통기한
def expiration_date():
    expiration_date_element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[4]/div[3]/span'))
    )
    expiration_date_text = expiration_date_element.text
    # print("유통기한:", expiration_date_text)
    df.at[idx, '유통기한'] = expiration_date_text
    return expiration_date_text

# 보관방법
def storage_method():
    storage_method_element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[1]/div[1]/div[2]/div/div[4]/div[4]/span'))
    )
    storage_method_text = storage_method_element.text
    # print("보관방법:", storage_method_text)
    df.at[idx, '보관방법'] = storage_method_text
    return storage_method_text

In [6]:
# 어울리는 안주
def all_side_dishes():
    try:
        # 부모 XPATH를 사용하여 모든 어울리는 안주들을 찾습니다.
        side_dishes_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[1]/div[4]/div[2]/div/*/label'))
        )
    
        # 각 안주의 텍스트를 리스트에 저장합니다.
        side_dishes_texts = [dish.text for dish in side_dishes_elements]
        # print("어울리는 안주:", side_dishes_texts)
        df.at[idx, '어울리는 안주'] = ", ".join(side_dishes)  # 여러 안주들을 문자열로 합쳐서 저장
        return side_dishes_texts
    
    except TimeoutException:
        dt.at[idx, '준비 중입니다.']
        print("어울리는 안주를 찾을 수 없습니다.")
        return []

In [7]:
for name in search_list:
    result = search(str(name))
    
    if result:
        result_element, found_drinks = result
        print(f"술 이름: {name}")
        print("주종:", type_liquor())
        print("도수:", number_alcohol())
        print("유통기한:", expiration_date())
        print("보관방법:", storage_method())
        
        side_dishes = all_side_dishes()
        print("어울리는 안주:", side_dishes)
        print("=" * 50)
        
print("Found drinks:")
print("총 {}개 입니다.".format(len(liquor_list)))
print(liquor_list)

검색 결과가 없습니다: 입장탁주
검색 결과가 없습니다: 톡 한잔 소주
술 이름: 자연담은 복분자막걸리


NameError: name 'idx' is not defined

In [ ]:
def compare_image(image1_file, image2_path):
    image1 = cv2.imread("image_file/prd/name")
    image2 = cv2.imread(image2_path)
    
    difference = cv2.subtract(image1, image2)
    non_zero_pixels = not cv2.countNonZero(differnece)
    
    if non_zero_pixels == 0:
        result = True
    else:
        result = False
    
    return result

image_dict = {
        0: os.path.join("C:\TeamProject\image", "icon_rating_zero.jpg"),
        1: os.path.join("C:\TeamProject\image", "icon_rating_one.jpg"),
        2: os.path.join("C:\TeamProject\image", "icon_rating_two.jpg"),
        3: os.path.join("C:\TeamProject\image", "icon_rating_three.jpg"),
        4: os.path.join("C:\TeamProject\image", "icon_rating_four.jpg"),
        5: os.path.join("C:\TeamProject\image", "icon_rating_five.jpg")
}
def get_taste_information():
    result_element
    parent_element = '//*[@id="__next"]/div/div[2]/div/div[1]/div[5]/div[2]/section'
    taste_elements = parent_element.find_elements_by_xpath('.//div[@class="taste-info"]')
    
    # 각 맛 정보와 이미지를 비교하여 맞는 사진을 찾고 점수를 매깁니다.
    taste_score = []
    for score, img_path in image_dict.items():
        for taste_element in taste_elements:
            if compare_image(img_path, taste_element) == True:
                taste_score.append((score, img_path))
                break
    print(test_score)
    
    return taste_score